In [2]:
import pulp

# Travel time matrix (in minutes) based on given data
travel_time = [
    [0, 3, 4, 5, 4, 5, 8, 10],
    [3, 0, 6, 4, 3, 5, 8, 7],
    [4, 6, 0, 3, 3, 2, 3, 2],
    [5, 4, 3, 0, 2, 3, 2, 2],
    [4, 3, 3, 2, 0, 3, 2, 2],
    [5, 5, 2, 3, 3, 0, 3, 4],
    [8, 8, 3, 2, 2, 3, 0, 2],
    [10, 7, 2, 2, 2, 4, 2, 0]
]

# Population of each district (in thousands)
population = [45, 55, 75, 60, 25, 30, 40, 40]

# Number of districts
num_districts = len(population)

# Set up the problem as a maximization problem
problem = pulp.LpProblem("Maximize_Coverage", pulp.LpMaximize)

# Decision variables: x[i] = 1 if an ambulance is placed in district i, otherwise 0
x = [pulp.LpVariable(f"x_{i}", cat="Binary") for i in range(num_districts)]

# Helper variables: coverage[i] = 1 if district i is covered by any ambulance within 2 minutes, otherwise 0
coverage = [pulp.LpVariable(f"coverage_{i}", cat="Binary") for i in range(num_districts)]

# Objective: Maximize the total population covered within two minutes
problem += pulp.lpSum([population[i] * coverage[i] for i in range(num_districts)]), "Total_Population_Covered"

# Coverage constraints: coverage[i] should be 1 if there is an ambulance within 2 minutes of district i
for i in range(num_districts):
    # Sum of x[j] for districts j within 2 minutes of district i must be >= coverage[i]
    problem += pulp.lpSum([x[j] for j in range(num_districts) if travel_time[i][j] <= 2]) >= coverage[i]

# Constraint: Total number of ambulances (change this for n = 1 or n = 2)
n = 2  # or change to 1 for testing n = 1
problem += pulp.lpSum(x) == n

# Solve the problem
solver = pulp.PULP_CBC_CMD()
problem.solve(solver)

# Display the results
print("Status:", pulp.LpStatus[problem.status])
print("Total Population Covered:", pulp.value(problem.objective))

# Display which districts have ambulances
for i in range(num_districts):
    print(f"District {i+1} - Ambulance: {int(x[i].value())}, Coverage: {int(coverage[i].value())}")

# Display population covered for each district
covered_population = sum(population[i] * int(coverage[i].value()) for i in range(num_districts))
print("Total Population Covered:", covered_population)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_r/6ppttwts2vn32pmq4vn9qt6w0000gn/T/6c266b6efe7840eca7d291533752e71c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_r/6ppttwts2vn32pmq4vn9qt6w0000gn/T/6c266b6efe7840eca7d291533752e71c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 95 RHS
At line 105 BOUNDS
At line 122 ENDATA
Problem MODEL has 9 rows, 16 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 295 - 0.00 seconds
Cgl0004I processed model has 9 rows, 14 columns (14 integer (13 of which binary)) and 30 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -295
Cbc0038I Cleaned solution of -295
C